<a href="https://colab.research.google.com/github/melzismn/Digital-Design-2020-2021/blob/master/LaplacianSmoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

In [ ]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans

# Read and visualize meshes

There are several libraries to do Geometry Processing in Python. In this case we will use the python binding of [libigl](https://libigl.github.io/libigl-python-bindings/). Another nice library for 3D visualization is [Open3D](http://www.open3d.org/).

In [ ]:
# Read and plot mesh
v, f = igl.read_triangle_mesh(os.path.join('.', "data", "bunny.off"))
H = igl.gaussian_curvature(v, f)
plot(v, f, H)

# **Computing the Laplacian and Mass Matrix**

Computing the cotangent matrix requires a bit of work (not really complicate, but boring). In this case we will use some ready methods.

In [ ]:
# Compute Laplacian and Area Matrix
L = -igl.cotmatrix(v, f)
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

invM = sp.sparse.linalg.inv(M)
Delta = sp.sparse.csc_matrix.todense(sp.sparse.csr_matrix.dot(invM,L)) 
L = np.matmul(np.diag(1/np.diag(Delta)),Delta)

# Laplacian smoothing

The algorithm is simple, please refer to the slides!

In [ ]:
# Coordinate after 5 iterations
n_iter = 5
alpha = 0.5

V = v
for i in range(n_iter):
    Vnew = V - alpha*np.matmul(L,V)
    V = Vnew
    
Hnew = igl.gaussian_curvature(Vnew, f)
plot(Vnew, f, Hnew)

In [ ]:
# Coordinate after 20 iterations
n_iter = 20
alpha = 0.5

V = v
for i in range(n_iter):
    Vnew = V - alpha*np.matmul(L,V)
    V = Vnew
    
Hnew = igl.gaussian_curvature(Vnew, f)
plot(Vnew, f, Hnew)